In [2]:
#imports
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


In [3]:
#dados
train = pd.read_csv('/content/drive/MyDrive/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/titanic/test.csv')

In [4]:
#separando por titulos
for df in [train, test]:
    df["Title"] = df["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip() if ',' in x and '.' in x.split(',')[1] else '')
    df["Title"] = df["Title"].replace(["Don", "Rev", "Dr", "Major", "Lady", "Sir", "Col", "Capt", "the Countess", "Jonkheer"], "Rare")
    df["Title"] = df["Title"].replace(["Mlle", "Ms"], "Miss")
    df["Title"] = df["Title"].replace(["Mme"], "Mrs")

In [5]:
#separando variaveis
target = 'Survived'
x_train, x_test, y_train, y_test = train_test_split(train.drop(target, axis=1), train[target], test_size=0.3, random_state=0)
df_train = pd.concat([x_train, y_train], axis=1)
df_test = pd.concat([x_test, y_test], axis=1)

In [6]:
#drop em colunas irrelevantes
for df in [df_train, df_test, test]:
    df.drop(['Name', 'Ticket'], axis=1, inplace=True, errors='ignore')

In [7]:
print('Número de valores únicos no conjunto de treinamento:')
print(df_train.nunique())
print('Número de valores únicos no conjunto de teste:')
print(df_test.nunique())

Número de valores únicos no conjunto de treinamento:
PassengerId    623
Pclass           3
Sex              2
Age             83
SibSp            7
Parch            7
Fare           216
Cabin          120
Embarked         3
Title            5
Survived         2
dtype: int64
Número de valores únicos no conjunto de teste:
PassengerId    268
Pclass           3
Sex              2
Age             64
SibSp            6
Parch            5
Fare           134
Cabin           48
Embarked         3
Title            5
Survived         2
dtype: int64


In [8]:
#separando dados categoricos e numericos
categorical_features = [col for col in df_train.columns if df_train[col].dtype == 'O']
numerical_features = [col for col in df_train.columns if df_train[col].dtype != 'O']
print('Features categóricas:\n', categorical_features)
print('Features numéricas:\n', numerical_features)

Features categóricas:
 ['Sex', 'Cabin', 'Embarked', 'Title']
Features numéricas:
 ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']


In [9]:
#remocao de outliers
for feature in numerical_features:
    if 0 in df_train[feature].unique():
        continue
    q1 = df_train[feature].quantile(0.25)
    q3 = df_train[feature].quantile(0.75)
    iqr = q3 - q1
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    df_train = df_train[(df_train[feature] < upper_limit) & (df_train[feature] > lower_limit)]

In [10]:
#preenchendo dados ausentes
for df in [df_train, df_test, test]:
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

for df in [df_test, test]:
    df['Fare'].fillna(df['Fare'].median(), inplace=True)


In [11]:
print(df_train.isnull().sum())
print(df_test.isnull().sum())
print(test.isnull().sum())

PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Cabin          359
Embarked         0
Title            0
Survived         0
dtype: int64
PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Cabin          216
Embarked         0
Title            0
Survived         0
dtype: int64
PassengerId      0
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Cabin          327
Embarked         0
Title            0
dtype: int64


In [12]:
# Codificar features categóricas
le = LabelEncoder()
for feature in categorical_features:
    for df in [df_train, df_test, test]:
        df[feature] = df[feature].astype(str)
    le.fit(pd.concat([df_train[feature], df_test[feature], test[feature]]))
    for df in [df_train, df_test, test]:
        df[feature] = le.transform(df[feature])

In [13]:
# Separar features e target
X_train = df_train.drop('Survived', axis=1)
y_train = df_train['Survived']
X_test = df_test.drop('Survived', axis=1)
y_test = df_test['Survived']

In [14]:
#fazendo as predições
model= GaussianNB()
model.fit(X_train, y_train)

y_pred= model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8097014925373134


In [15]:
precision= precision_score(y_test, y_pred)
print('Precisão:', precision)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity= tn/(tn+fp)
print('Especificidade:', specificity)

recall= recall_score(y_test, y_pred)
print('Recall:', recall)

f1= f1_score(y_test, y_pred)
print('F1-score:', f1)


Precisão: 0.7474747474747475
Especificidade: 0.8511904761904762
Recall: 0.74
F1-score: 0.743718592964824


In [16]:
X_train = X_train.drop('Cabin', axis=1)
model.fit(X_train, y_train)
test_predictions = model.predict(test.drop('Cabin', axis=1))

submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': test_predictions})

submission.to_csv('titanicNVB.csv', index=False)